In [ ]:
M8 = {
    'width': 11.4,
    'length':110,
    'height': 4,
    'height_loaded': 8,
    'emissionfactor': 0.7,
    'power': 318,
    'own_weight': 100,
    'loaded_capacity':2711,
    'capacity_unloaded':0,
    'loaded_speed': 3.7777,
    'speed_unloaded': 5.3333,
    'loaded_resistance': 106453,
    'resistance_unloaded':61526,
    # variable values depend on random variables
    'is_loaded': False,
}

M8_loaded = {
    'width': 11.4,
    'length':110,
    'height': 4,
    'height_loaded': 8,
    'emissionfactor': 0.7,
    'power': 318,
    'own_weight': 100,
    'loaded_capacity':2711,
    'capacity_unloaded':0,
    'loaded_speed': 3.7777,
    'speed_unloaded': 5.3333,
    'loaded_resistance': 106453,
    'resistance_unloaded':61526,
    # variable values depend on random variables
    'is_loaded': True,
}

C4 = {
    'width': 22.8,
    'length':185,
    'height': 3,
    'height_loaded': 8,
    'emissionfactor': 0.71,
    'power': 400,
    'own_weight':150,
    'loaded_capacity':7250,
    'capacity_unloaded':0,
    'loaded_speed': 3.00,
    'speed_unloaded':4.472,
    'loaded_resistance': 111004,
    'resistance_unloaded':68736,
    # variable values depend on random variables
    'is_loaded': False,
}


C4_loaded = {
    'width': 22.8,
    'length':185,
    'height': 3,
    'height_loaded': 8,
    'emissionfactor': 0.71,
    'power': 400,
    'own_weight':150,
    'loaded_capacity':7250,
    'capacity_unloaded':0,
    'loaded_speed': 3.00,
    'speed_unloaded':4.472,
    'loaded_resistance': 111004,
    'resistance_unloaded':68736,
    # variable values depend on random variables
    'is_loaded': True,
}


BII_4 = {
    'width': 22.8,
    'length':195,
    'height': 3,
    'height_loaded': 8,
    'emissionfactor': 0.71,
    'power': 400,
    'own_weight': 160,
    'loaded_capacity':12000,
    'capacity_unloaded':0,
    'loaded_speed': 2.833,
    'speed_unloaded':4.611,
    'loaded_resistance': 861913,
    'resistance_unloaded':114953,
    # variable values depend on random variables
    'is_loaded': False,
}

BII_4_loaded = {
    'width': 22.8,
    'length':195,
    'height': 3,
    'height_loaded': 8,
    'emissionfactor': 0.71,
    'power': 400,
    'own_weight': 160,
    'loaded_capacity':12000,
    'capacity_unloaded':0,
    'loaded_speed': 2.833,
    'speed_unloaded':4.611,
    'loaded_resistance': 861913,
    'resistance_unloaded':114953,
    # variable values depend on random variables
    'is_loaded': True,
}


ships = [M8_loaded, M8, BII_4, BII_4_loaded,C4, C4_loaded]

# For testing purposes we only want v to be 1
def compute_v_provider(v_empty, v_full):
    return lambda x: (v_empty + v_full) / 2

TransportResource = type('TransportResource', 
                         (core.Identifiable, core.Log, core.ContainerDependentMovable, 
                          core.HasResource, core.Routeable), {})
                
const_data_vessel = {"env": None,
               "name": "Vessel",
               "route": None,
               "geometry": shapely.geometry.Point(0, 0),  # lon, lat
               "capacity": 1_000,
               "compute_v": compute_v_provider(v_empty=0.05, v_full=0.05)}

In [ ]:
def init_random_vessel(routes, vessel_name):
    vessel_dict = random.choice(ships)
    # init_vessels gives list but need just one
    vessel = init_vessels([vessel_dict], routes)[0]
    vessel.name = vessel_name
    vessel.shipType = 'None'
    return vessel



     #Normal situation
def init_vessels(list_vessel_dict, routes):    
    correction = 0
    if random.random() < 0.35:
        correction = 8.7
    vessels = []
    height = 0
    for i in range(len(list_vessel_dict)):
        for path in routes:
            if path.closed:
                continue
            if list_vessel_dict[i]["is_loaded"]:
                height = list_vessel_dict[i]["height_loaded"]
                if list_vessel_dict[i]["height_loaded"] >= path.max_height or correction >= path.max_height:
                    continue
            else:
                height = list_vessel_dict[i]["height"]
                if list_vessel_dict[i]["height"] >= path.max_height or correction >= path.max_height:
                    continue
            break
        vessel = TransportResource(**const_data_vessel)
        vessel.height = height
        vessel.length = list_vessel_dict[i]["length"]
        vessel.route = path.nodes
        vessel.loaded = list_vessel_dict[i]["is_loaded"]
        if vessel.loaded:
            speed = list_vessel_dict[i]["loaded_speed"]
            vessel.compute_v = compute_v_provider(speed, speed)
            vessel.resistance = list_vessel_dict[i]["loaded_resistance"]
        else:
            speed = list_vessel_dict[i]["speed_unloaded"]
            vessel.compute_v = compute_v_provider(speed, speed)
            vessel.resistance =list_vessel_dict[i]["resistance_unloaded"]
        vessel.speed = speed
        vessel.emission = list_vessel_dict[i]["emissionfactor"]
        vessel.name = "Vessel " + str(i + 1)
        vessel.path_complete = path
        vessel.loaded_capacity = list_vessel_dict[i]["loaded_capacity"]
        # 50/50 choses different direction
        if random.random() < 0.5:
            vessel.route = vessel.route[::-1]
            vessel.path_complete.nodes = vessel.path_complete.nodes[::-1]
        vessels.append(vessel)
    
    return vessels